In [13]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv("/users/s1f102201582/projects/mhcc-moshi/.env")

True

In [14]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,4"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1000

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = True

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/data_stereo")
audio_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/data_stereo")

# mfa関連のパス
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")
mfa_input_dir = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/mfa_input")
mfa_output_dir = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/mfa_output")

#RAGで読み取るPDFのパス
rag_pdf_dir = join(home_dir, "projects/mhcc-moshi/mental_docs/")

In [15]:
base_paths = [
    json_dir_path,
    audio_dir_path,
    mfa_input_dir,
    mfa_output_dir,
]

for p in base_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [16]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [17]:
loader = DirectoryLoader(
    rag_pdf_dir,
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                     | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set g

Loaded 3 documents


In [18]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [19]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [20]:
from typing import Literal
from pydantic import BaseModel, Field

class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    raw_text: str = Field(..., description="FishAudioタグなしの話者が話した内容。")
    tag_text: str = Field(..., description="FishAudioタグありの話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [21]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [22]:
#promptを作成
import random

prompt_template = """あなたは、Fish Audioによる音声合成（TTS）用の対話シナリオを作成するプロのライターです。
以下の【要件】、【設定】、そして【指示】に従い、対話データ（Dialogues）を生成してください。

### 【目的】
メンタルヘルスケアのカウンセリング対話をシミュレーションします。

### 【出力データ構造の定義】
* **speaker**: "A" (カウンセラー) または "B" (クライエント)
* **tag_text**: 文頭に必ずFish Audio用マーカー（例: `(sad)`) を付与したテキスト。
* **raw_text**: `tag_text`からマーカーを削除した純粋なテキスト。

### 【最重要要件：自然な会話の再現】
1.  **短いターン:** 一度の発話は短く区切る。長い独白は禁止。
2.  **インターラプト:** Bが長く話そうとしたら、Aが「(listening)うんうん」と短い相槌で挟む。
3.  **タグの使用:** 文頭に必ずタグをつける。

### 【禁止事項】
* **プレースホルダーの禁止:** テキスト中に「〇〇さん」「××さん」のような伏せ字・プレースホルダーを含めることは**厳禁**です。
* **カウンセラーの「知ったかぶり」禁止:** カウンセラー(A)が、クライアント(B)が口にする前に、前回の内容や宿題について言及することを禁止します。文脈は必ずBの発言によって作ってください。

### 【指示】

1.  **会話の開始と導入フェーズ（最重要）:**
    * **基本ルール:** カウンセラー(A)は、クライアント(B)の現在の状況や前回の詳細を**忘れている/知らない**ものとして振る舞ってください。
    
    * **状況が「初回」の場合:**
        * A: まず**名前**を尋ねる。 -> B: **具体的な苗字**（佐藤、鈴木など）を名乗る。
        * A: 「今日はどのようなことでお見えになりましたか？」と**来談理由**を尋ねる。
        * B: 指定された**【トピック】**についての悩みを話し始める。
        
    * **状況が「初回」以外の場合:**
        * A: 名前は呼ばずに挨拶し、「前回からいかがですか？」や「今日はどのようなお話をしましょうか？」と**完全にオープンな質問**をする。（※「宿題はどうでしたか？」などとAから特定の話題を振ることは禁止）。
        * B: Aの質問に答える形で、**【トピック】**や**【状況】**（宿題があったことや、前回の続きなど）を**自分から説明する**。
        * A: Bの説明を聞いて初めて、「ああ、そうでしたね」や「なるほど、そういう状況なのですね」と状況を把握・確認する。

    * **共通事項:**
        * AがBの話を聞いて状況を把握した**後**に、初めて具体的なカウンセリングへ移行してください。
        * 導入のヒアリング段階であっても、「短いターン」と「インターラプト」のルールは守ってください。

2.  **設定:**
    * クライアント(B)のペルソナ: {selected_persona}
    * トピック: {selected_topic}
    * 状況: {selected_situation}
    * 話し方指示: {selected_style_instruction}

3.  **会話の構成:**
    * 会話量は **4分程度** を目安とし、**25〜35往復（ターン）** 程度生成してください。
    * ペルソナの背景（年齢・職業）を反映させる（直接言わせず、内容で匂わせる）。
    * クライアント(B)の話し方指示（よどみ具合など）を忠実に守る。

出力は指定された `Dialogues` スキーマに従ってください。
"""

# トピックリスト（悩みの種類）(全40項目)
topic_list = [
  # 仕事・キャリア関連
  "仕事のプレッシャーや過労、バーンアウト",
  "職場の人間関係（上司、同僚、部下）",
  "キャリアプランの悩み、キャリアチェンジの不安",
  "転職・就職活動のストレス",
  "ハラスメント（パワハラ、モラハラなど）の影響",
  "仕事へのモチベーション低下、やる気が出ない",

  # 自己認識・感情関連
  "自己肯定感の低さ、自分を責めてしまう",
  "完璧主義、失敗への過度な恐れ",
  "他人の評価が過度に気になる、承認欲求",
  "劣等感（他人との比較）",
  "自分のやりたいことが分からない、アイデンティティの悩み",
  "感情のコントロールが難しい（怒り、イライラ、悲しみ）",
  "ネガティブ思考の癖、反芻思考（同じことをぐるぐる考えてしまう）",
  "焦燥感、何かに追われている感覚",
  "罪悪感（休むことへの罪悪感など）",
  "虚無感、むなしさ、生きがいを感じられない",
  "趣味や楽しみを感じられない（アンヘドニア）",
  
  # 対人関係
  "家族関係（親子、夫婦、兄弟、親戚）",
  "友人関係や恋愛関係の悩み",
  "コミュニケーションへの苦手意識（雑談、会議での発言など）",
  "人に頼ることができない、甘えられない",
  "他人の期待に応えすぎようとしてしまう（ピープルプリーザー）",
  "境界線（バウンダリー）の問題（NOと言えない）",
  "孤独感、疎外感",
  "HSP（繊細さ）に関する悩み",
  
  # 生活・健康関連
  "将来への漠然とした不安",
  "気分の落ち込み、無気力",
  "睡眠に関する悩み（寝付けない、途中で起きる、過眠）",
  "生活リズムの乱れ",
  "体調不良（頭痛、倦怠感、腹痛など）と気分の関連",
  "健康不安（病気への過度な心配）",

  # 習慣・行動関連
  "決断疲れ、何かを選ぶことができない",
  "先延ばし癖、物事が始められない",
  "SNS疲れ、デジタルデトックスの必要性",

  # 特定の出来事
  "ライフイベント（引っ越し、結婚、出産、育児、介護）に伴うストレス",
  "特定の出来事による軽いトラウマやフラッシュバック",
  "喪失体験（別れ、死別）からの回復（グリーフケア）",
  "過去の選択への後悔"
]

# 状況リスト（セッションの場面）
situation_list = [
  "初回",
  "初期",
  "中期（深掘り）",
  "中期（宿題の確認）",
  "中期（感情の表出）",
  "後期",
  "終了（終結）"
]

# クライアントのペルソナリスト（年齢と職業/立場のみ）
persona_list = [
    "20代・会社員",
    "30代・会社員",
    "40代・会社員",
    "50代・会社員",
    "20代・大学生",
    "20代・大学院生",
    "30代・管理職",
    "40代・主婦/主夫",
    "30代・フリーランス",
    "20代・アルバイト",
    "40代・パートタイム",
    "50代・自営業",
    "20代・求職中",
    "30代・育児休業中"
]

# Fish Audio用マーカーリスト (プロンプト埋め込み用)
fish_audio_markers = """
【基本感情】
(angry) (sad) (excited) (surprised) (satisfied) (delighted) 
(scared) (worried) (upset) (nervous) (frustrated) (depressed) 
(empathetic) (embarrassed) (disgusted) (moved) (proud) (relaxed) 
(grateful) (confident) (interested) (curious) (confused) (joyful)

【高度な感情】
(disdainful) (unhappy) (anxious) (hysterical) (indifferent) 
(impatient) (guilty) (scornful) (panicked) (furious) (reluctant) 
(keen) (disapproving) (negative) (denying) (astonished) (serious) 
(sarcastic) (conciliative) (comforting) (sincere) (sneering) 
(hesitating) (yielding) (painful) (awkward) (amused)

【トーン】
(in a hurry tone) (shouting) (screaming) (whispering) (soft tone)

【特殊効果】
(laughing) (chuckling) (sobbing) (crying loudly) (sighing) 
(panting) (groaning) (crowd laughing) (background laughter) (audience laughing)
"""

# 話し方指示（Fish Audioタグ指定付き）
speaking_style_data = {
    "よどみが多い（ためらいがち）": "話者Bの発話において、`(hesitating)` `(sighing)` などのタグや、フィラー（「えーと」「あのー」）を多用し、言葉に詰まる様子を表現してください。",
    "普通（自然な会話）": "文脈に合わせて適切な【基本感情】タグを文頭に付与し、自然な会話の範囲でフィラーを含めてください。",
    "スムーズ（よどみ少なめ）": "話者Bの発話は流暢にし、`(confident)` `(serious)` などのタグを用いて、しっかりとした口調を表現してください。"
}

def gen_prompt_txt():
    # ランダムに選択
    selected_topic = random.choice(topic_list)
    selected_situation = random.choice(situation_list)
    selected_persona = random.choice(persona_list)
    selected_style_name = random.choice(list(speaking_style_data.keys()))
    selected_style_instruction = speaking_style_data[selected_style_name]

    print("選ばれたトピック:", selected_topic)
    print("選ばれた状況:", selected_situation)
    print("選ばれたペルソナ:", selected_persona)
    print("選ばれたクライアントのスタイル:", selected_style_name)

    # 変数をプロンプトに埋め込む
    final_prompt = prompt_template.format(
        selected_persona=selected_persona,
        selected_topic=selected_topic,
        selected_situation=selected_situation,
        selected_style_name=selected_style_name,
        selected_style_instruction=selected_style_instruction,
        fish_audio_markers=fish_audio_markers
    )
    
    return final_prompt

In [25]:
# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [28]:
dialogues_list = gen_txt_dialogue()
print("\n")
for dial in dialogues_list:
    print(f"{dial.speaker}: {dial.tag_text}")

選ばれたトピック: 特定の出来事による軽いトラウマやフラッシュバック
選ばれた状況: 初期
選ばれたペルソナ: 50代・自営業
選ばれたクライアントのスタイル: よどみが多い（ためらいがち）


A: (gentle)こんにちは。今日はどのようなことでお見えになりましたか？
B: (hesitating)あのー、田中と申します。最近ちょっと、えー、困っていることがありまして...
A: (calm)田中さん、そうでしたか。具体的には、どのようなことでしょうか？
B: (sighing)ええと、半年前くらいに、ちょっと、仕事で大きな問題がありまして。それから、時々、あの時のことを思い出して、急に不安になるんです。
A: (listening)うんうん。
B: (nervous)特に、その、お金関係の計算をしている時とかに、急に、えー、動悸がしてしまって。
A: (calm)お金関係の計算をしている時に、動悸がするのですね。
B: (hesitating)はい。あの時、本当に、えー、自営業なので、もう全てを失うんじゃないかと...思ってしまって。
A: (listening)うんうん。
B: (sighing)それが、急に、こう、フラッシュバックするような感じで、心臓がドキドキして、手も震えてきたりするんです。
A: (understanding)フラッシュバックして、心臓がドキドキして、手も震えるのですね。
B: (nervous)ええ。その、何て言うか、もう、息苦しくなってしまうこともあって...
A: (empathic)息苦しくなるほど、ですか。それは、大変お辛いですね。
B: (worried)はい...もう、どうしていいか分からなくて...
A: (calm)その動悸や手の震えは、具体的にどのような状況で起こることが多いですか？
B: (thinking)ええと、例えば、パソコンで経理ソフトを開いて、請求書を入力している時とか。
A: (listening)うんうん。
B: (hesitating)あとは、あのー、銀行の通帳を記帳しに行った時なんかも、ちょっと、足がすくんでしまったり。
A: (understanding)経理ソフトの入力中や、銀行で記帳する時ですね。
B: (sighing)はい。普段は何でもないことなのに、急に、こう、あの時の嫌な感じが蘇